In [ ]:
#!/usr/bin/env python3

from mastodon import Mastodon, StreamListener
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
#API_BASE = 'https://dogukankefeli.tech'
class Pure(Mastodon):
    def __init__(self,access_token = 'secrets/usercred.secret',
            api_base_url = 'https://mastodon.social'):
        super().__init__(access_token = 'secrets/usercred.secret',
            api_base_url = 'https://mastodon.social')
    def title_and_desc(self):
        size = 'width="20" height="20"'
        instance_logs = self.instance()
        title = instance_logs['uri']
        note = instance_logs['short_description']
        desc = note[:note.find('/>')]+size+note[note.find('/>'):]
        return title, desc
    
    def get_timeline_users(self):
        timeline = self.timeline()
        user = []
        for i in range(len(timeline)):
            user.append(timeline[i]['account']['id'])
        self.unique = np.unique(user)
    
    def create_df(self):
        user_ids = []
        usernames = []
        toot_ids = []
        toot_time = []
        toot_favourites_count = []
        content = []
        for user in self.unique:
            statuses = self.account_statuses(user)
            for toot in statuses:
                user_ids.append(toot['account']['id'])
                usernames.append(toot['account']['username'])
                toot_ids.append(toot['id'])
                toot_time.append(toot['created_at'])
                toot_favourites_count.append(toot['favourites_count'])
                soup = BeautifulSoup(toot['content'],features="lxml")
                content.append(soup.get_text())
        data = {}
        data['user_ids'] = user_ids
        data['usernames'] = usernames
        data['toot_ids'] = toot_ids
        data['toot_time'] = toot_time
        data['favourites_count'] = toot_favourites_count
        data['content']=content
        df = pd.DataFrame(data)
        return df

    def get_user_id(self,name):
        """Searches the server for given name and returns the id number of first result""" 
        search_results = self.account_search(name)
        id = search_results[0]['id']
        return id
    
    def get_user_toots(self,id):
        """Returns a dataframe of toots for given user id"""
        username = []
        user_ids = []
        toot_ids = []
        toot_time = []
        toot_favourites_count = []
        content = []
        statuses = self.account_statuses(id,limit=40)
        for toot in statuses:
            user_ids.append(id)
            username.append(toot['account']['username'])
            toot_ids.append(toot['id'])
            toot_time.append(toot['created_at'])
            toot_favourites_count.append(toot['favourites_count'])
            soup = BeautifulSoup(toot['content'],features="lxml")
            content.append(soup.get_text())
        data = {}
        data['user_ids'] = user_ids
        data['username'] = username
        data['toot_ids'] = toot_ids
        data['toot_time'] = toot_time
        data['favourites_count'] = toot_favourites_count
        data['content']=content
        df = pd.DataFrame(data)
        return df


    def timeline_network(self,id=958614):
        
        timeline = self.timeline(limit=200)
        user = {}
        for toot in timeline:
          user[toot['account']['id']] = toot['account']['username']
        source = [] #users
        target = [] #followers

        for key, value in user.items():
          u_follower = self.account_followers(key)
          for k in u_follower:
              source.append(value)
              target.append(k['username'])
        df = pd.DataFrame()
        df['Source'] = source
        df['Target'] = target
        df['Weight'] = 0.1
        return df

    def followings_network(self,id=958614):
        username = self.account(id)['username']

        followings = self.account_following(id)
        users={}
        source = [] #users
        target = [] #followers
        for following in followings:
            users[following['id']]=following['username']
            source.append(username)
            target.append(following['username'])

        for key, value in users.items():
            u_follower = self.account_following(key)
            for k in u_follower:
                source.append(value)
                target.append(k['username'])
        df = pd.DataFrame()
        df['Source'] = source
        df['Target'] = target
        df['Weight'] = 0.1
        df.loc[df['Source']==username,'Weight']=0.2
        return df



    def account_info(self,id=106555351749444654):

        acc_dict = self.account(id)
        smtxt="""
        <html lang="en" ng-app="myApp" class="ng-scope"><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><style type="text/css">[uib-typeahead-popup].dropdown-menu{display:block;}</style><style type="text/css">.uib-time input{width:50px;}</style><style type="text/css">[uib-tooltip-popup].tooltip.top-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.top-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-left > .tooltip-arrow,[uib-tooltip-popup].tooltip.bottom-right > .tooltip-arrow,[uib-tooltip-popup].tooltip.left-top > .tooltip-arrow,[uib-tooltip-popup].tooltip.left-bottom > .tooltip-arrow,[uib-tooltip-popup].tooltip.right-top > .tooltip-arrow,[uib-tooltip-popup].tooltip.right-bottom > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.top-left > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.top-right > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.bottom-left > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.bottom-right > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.left-top > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.left-bottom > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.right-top > .tooltip-arrow,[uib-tooltip-html-popup].tooltip.right-bottom > .tooltip-arrow,[uib-tooltip-template-popup].tooltip.top-left > .tooltip-arrow,[uib-tooltip-template-popup].tooltip.top-right > .tooltip-arrow,[uib-tooltip-template-popup].tooltip.bottom-left > .tooltip-arrow,[uib-tooltip-template-popup].tooltip.bottom-right > .tooltip-arrow,[uib-tooltip-template-popup].tooltip.left-top > .tooltip-arrow,[uib-tooltip-template-popup].tooltip.left-bottom > .tooltip-arrow,[uib-tooltip-template-popup].tooltip.right-top > .tooltip-arrow,[uib-tooltip-template-popup].tooltip.right-bottom > .tooltip-arrow,[uib-popover-popup].popover.top-left > .arrow,[uib-popover-popup].popover.top-right > .arrow,[uib-popover-popup].popover.bottom-left > .arrow,[uib-popover-popup].popover.bottom-right > .arrow,[uib-popover-popup].popover.left-top > .arrow,[uib-popover-popup].popover.left-bottom > .arrow,[uib-popover-popup].popover.right-top > .arrow,[uib-popover-popup].popover.right-bottom > .arrow,[uib-popover-html-popup].popover.top-left > .arrow,[uib-popover-html-popup].popover.top-right > .arrow,[uib-popover-html-popup].popover.bottom-left > .arrow,[uib-popover-html-popup].popover.bottom-right > .arrow,[uib-popover-html-popup].popover.left-top > .arrow,[uib-popover-html-popup].popover.left-bottom > .arrow,[uib-popover-html-popup].popover.right-top > .arrow,[uib-popover-html-popup].popover.right-bottom > .arrow,[uib-popover-template-popup].popover.top-left > .arrow,[uib-popover-template-popup].popover.top-right > .arrow,[uib-popover-template-popup].popover.bottom-left > .arrow,[uib-popover-template-popup].popover.bottom-right > .arrow,[uib-popover-template-popup].popover.left-top > .arrow,[uib-popover-template-popup].popover.left-bottom > .arrow,[uib-popover-template-popup].popover.right-top > .arrow,[uib-popover-template-popup].popover.right-bottom > .arrow{top:auto;bottom:auto;left:auto;right:auto;margin:0;}[uib-popover-popup].popover,[uib-popover-html-popup].popover,[uib-popover-template-popup].popover{display:block !important;}</style><style type="text/css">.uib-datepicker-popup.dropdown-menu{display:block;float:none;margin:0;}.uib-button-bar{padding:10px 9px 2px;}</style><style type="text/css">.uib-position-measure{display:block !important;visibility:hidden !important;position:absolute !important;top:-9999px !important;left:-9999px !important;}.uib-position-scrollbar-measure{position:absolute !important;top:-9999px !important;width:50px !important;height:50px !important;overflow:scroll !important;}.uib-position-body-scrollbar-measure{overflow:scroll !important;}</style><style type="text/css">.uib-datepicker .uib-title{width:100%;}.uib-day button,.uib-month button,.uib-year button{min-width:100%;}.uib-left,.uib-right{width:100%}</style><style type="text/css">.ng-animate.item:not(.left):not(.right){-webkit-transition:0s ease-in-out left;transition:0s ease-in-out left}</style><style type="text/css">@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}.ng-animate-shim{visibility:hidden;}.ng-anchor{position:absolute;}</style>
        """
        full_text = """
          <!--<base href="/">--><base href=".">
          <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/css/bootstrap.min.css">
          <link rel="stylesheet" href="https://botometer.osome.iu.edu/static/bower_components/bootstrap/dist/css/bootstrap.min.css">
          <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.0/js/bootstrap.min.js"></script>
          <link rel="stylesheet" href="https://botometer.osome.iu.edu/static/app.css">
        </head>
        <body>
            <div role="main" class="container">
                <!-- ngView: --><div ng-view="" class="ng-scope"><h1 class="user-detail-screen-name ng-binding ng-scope">
              @{username}
              <icon-verified user="user" class="ng-isolate-scope"><!-- ngIf: user.userData.verified --></icon-verified>
            </h1>
            <a class="user-detail-profile-link ng-scope" ng-href="https://mastodon.social/@{username}" href="https://mastodon.social/@{username}">
              <i class="glyphicon glyphicon-user"></i>
            </a>
            <div class="user-detail-banner-container ng-scope">
              <img class="user-detail-banner-img" ng-src="{header_static}" src="{header_static}">
              <img class="user-detail-profile-img img-circle" ng-src="{avatar_static}" src="{avatar_static}">
            </div>
            <div class="row ng-scope">
              <div class="col-sm-8 col-md-6">
                <dl class="dl-horizontal">
                  <dt>Screen name</dt>
                  <dd ng-show="user.userData" class="ng-binding">@{username}</dd>
                  <dt>Display name</dt>
                  <dd class="ng-binding">{display_name}</dd>
                  <dt>Description</dt>
                  <dd class="ng-binding">{note}</dd>
                  <dt>URL</dt>
                  <dd><a href="{url}">{url}</a></dd>
                </dl>
              </div>
              <div class="col-sm-4">
                <dl class="dl-horizontal">
                  <dt>Toots</dt>
                  <dd class="ng-binding">{statuses_count}</dd>
                  <dt>Following</dt>
                  <dd class="ng-binding">{following_count}</dd>
                  <dt>Followers</dt>
                  <dd class="ng-binding">{followers_count}</dd>
                </dl>
              </div>
            </div>
        <dl class="dl-horizontal ng-scope">
            <dt>Mastodon user ID</dt>
            <dd class="ng-binding">{id}</dd>
        </dl>
        </div>
          </div>
        """.format(id=acc_dict['id'],header_static=acc_dict['header_static'],avatar_static=acc_dict['avatar_static'],username=acc_dict['username'],display_name=acc_dict['display_name'],note=acc_dict['note'],url=acc_dict['url'],statuses_count=acc_dict['statuses_count'],following_count=acc_dict['following_count'],followers_count=acc_dict['followers_count'])

        Html_file = open("profile.html","w")
        Html_file.write(full_text)
        Html_file.close()

In [1]:
from pure import Pure

In [2]:
pure = Pure()

In [3]:
pure.instance()

{'uri': 'mastodon.social',
 'title': 'Mastodon',
 'short_description': 'Server run by the main developers of the project <img draggable="false" alt="🐘" class="emojione" src="https://mastodon.social/emoji/1f418.svg" /> It is not focused on any particular niche interest - everyone is welcome as long as you follow our code of conduct!',
 'description': 'Server run by the main developers of the project <img draggable="false" alt="🐘" class="emojione" src="https://mastodon.social/emoji/1f418.svg" /> It is not focused on any particular niche interest - everyone is welcome as long as you follow our code of conduct!',
 'email': 'staff@mastodon.social',
 'version': '3.4.1',
 'urls': {'streaming_api': 'wss://mastodon.social'},
 'stats': {'user_count': 565875,
  'status_count': 30910616,
  'domain_count': 19554},
 'thumbnail': 'https://files.mastodon.social/site_uploads/files/000/000/001/original/vlcsnap-2018-08-27-16h43m11s127.png',
 'languages': ['en'],
 'registrations': True,
 'approval_require

In [72]:
import pandas as pd
minid=0
followings = pure.account_following(149988)
df = pd.DataFrame(followings)
more_followings = []
for _ in range(10):
    minid = df['id'].max()
    followings = pure.account_following(149988,min_id=minid)
    more_followings+=followings
    df2 = pd.DataFrame(followings)
    df = df.append(df2,ignore_index=True)
df

,id,username,acct,display_name,locked,bot,discoverable,group,created_at,note,...,avatar,avatar_static,header,header_static,followers_count,following_count,statuses_count,last_status_at,emojis,fields
0,106704871185429094,leliboli,leliboli@polyglot.city,Veronica,True,False,False,False,2021-07-24 00:00:00+00:00,<p>en / ru / de / (sv) / (fr)</p>,...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/h...,https://files.mastodon.social/cache/accounts/h...,94,21,0,NaT,[],"[{'name': 'Pronouns', 'value': 'she/her', 'ver..."
1,106702995595872553,TQ,TQ@weirder.earth,TQ 🎹,True,False,True,False,2021-08-05 00:00:00+00:00,<p>Jackfruit of all trades. Old as in Walkman....,...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/h...,https://files.mastodon.social/cache/accounts/h...,709,56,158,2021-08-07,[],"[{'name': 'Pronouns', 'value': 'she/her | they..."
2,106694427751003020,gamehawk,gamehawk@weirder.earth,Karen C,False,False,True,False,2021-08-03 00:00:00+00:00,<p>🥨 👵 🌲 🏖️ On the edge of the Pine Barrens be...,...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/h...,https://files.mastodon.social/cache/accounts/h...,270,109,22,2021-08-06,[],"[{'name': 'Art account', 'value': '<span class..."
3,1307066,tim,tim@libranet.de,Tim,True,False,True,False,2020-10-30 00:00:00+00:00,Interested in Consciousness Exploration. I pla...,...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/a...,https://mastodon.social/headers/original/missi...,https://mastodon.social/headers/original/missi...,22,63,77,2021-07-30,[],[]
4,106608769254265505,simon,simon@writing.exchange,,False,False,False,False,2021-07-19 00:00:00+00:00,<p></p>,...,https://mastodon.social/avatars/original/missi...,https://mastodon.social/avatars/original/missi...,https://mastodon.social/headers/original/missi...,https://mastodon.social/headers/original/missi...,1,4,7,2021-07-28,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,1281624,RTP,RTP@fosstodon.org,(RTP) Privacy & Tech Tips,False,False,True,False,2020-09-15 00:00:00+00:00,"<p>Topics: <a href=""https://fosstodon.org/tags...",...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/h...,https://files.mastodon.social/cache/accounts/h...,401,94,1335,2021-08-07,[],[]
436,344630,skalman,skalman@mastodon.acc.umu.se,boring middle-aged man,True,False,False,False,2018-04-30 00:00:00+00:00,<p>Occasional hacker. Dad. Boring. Personal an...,...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/h...,https://files.mastodon.social/cache/accounts/h...,106,345,2123,2021-08-06,[],"[{'name': 'Essentials', 'value': 'Markdown, Pa..."
437,1350089,th4,th4@post.lurk.org,th4,False,False,False,False,2020-12-30 00:00:00+00:00,<p>I'm a musician working with TidalCycles and...,...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/h...,https://files.mastodon.social/cache/accounts/h...,65,72,333,2021-08-05,[],"[{'name': 'Website', 'value': '<a href=""https:..."
438,402879,laura,laura@mastodon.laurakalbag.com,Laura Kalbag,False,False,False,False,2018-08-02 00:00:00+00:00,"<p>½ of Small Technology Foundation. Designs, ...",...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/h...,https://files.mastodon.social/cache/accounts/h...,2618,222,990,2021-07-28,[],"[{'name': 'Small Technology website', 'value':..."


In [73]:
df.loc[df['bot']==True]

,id,username,acct,display_name,locked,bot,discoverable,group,created_at,note,...,avatar,avatar_static,header,header_static,followers_count,following_count,statuses_count,last_status_at,emojis,fields
7,106487135596068032,MeeXtina,MeeXtina@writing.exchange,Truth be told,False,True,True,False,2020-09-29 00:00:00+00:00,<p>A girl who loves frogs and many other thing...,...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/h...,https://files.mastodon.social/cache/accounts/h...,5,6,84,2021-07-06,[],"[{'name': 'Poetry', 'value': 'Quotes', 'verifi..."
10,106263435152923293,fedivision,fedivision@botsin.space,Official FediVision bot,False,True,False,False,2021-05-19 00:00:00+00:00,"<p>This is the official bot for the <a href=""h...",...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/a...,https://mastodon.social/headers/original/missi...,https://mastodon.social/headers/original/missi...,7,1,86,2021-05-30,[],"[{'name': 'Bot by', 'value': '<span class=""h-c..."
22,1203192,wikipediahaiku,wikipediahaiku@botsin.space,Wikipedia Haiku,False,True,False,False,2020-04-29 00:00:00+00:00,<p>Sentences from Wikipedia pages that acciden...,...,https://files.mastodon.social/accounts/avatars...,https://files.mastodon.social/accounts/avatars...,https://files.mastodon.social/accounts/headers...,https://files.mastodon.social/accounts/headers...,261,3,2786,2021-08-07,[],[]
29,1132330,biodiversitypix,biodiversitypix@botsin.space,Biodiversity Illustrations,False,True,True,False,2020-02-29 00:00:00+00:00,<p>The Biodiversity Heritage Library has thous...,...,https://files.mastodon.social/accounts/avatars...,https://files.mastodon.social/accounts/avatars...,https://mastodon.social/headers/original/missi...,https://mastodon.social/headers/original/missi...,1235,1,11704,2021-08-07,[],"[{'name': 'Creator', 'value': 'Mark Sample', '..."
47,106487135596068032,MeeXtina,MeeXtina@writing.exchange,Truth be told,False,True,True,False,2020-09-29 00:00:00+00:00,<p>A girl who loves frogs and many other thing...,...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/h...,https://files.mastodon.social/cache/accounts/h...,5,6,84,2021-07-06,[],"[{'name': 'Poetry', 'value': 'Quotes', 'verifi..."
50,106263435152923293,fedivision,fedivision@botsin.space,Official FediVision bot,False,True,False,False,2021-05-19 00:00:00+00:00,"<p>This is the official bot for the <a href=""h...",...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache/accounts/a...,https://mastodon.social/headers/original/missi...,https://mastodon.social/headers/original/missi...,7,1,86,2021-05-30,[],"[{'name': 'Bot by', 'value': '<span class=""h-c..."
62,1203192,wikipediahaiku,wikipediahaiku@botsin.space,Wikipedia Haiku,False,True,False,False,2020-04-29 00:00:00+00:00,<p>Sentences from Wikipedia pages that acciden...,...,https://files.mastodon.social/accounts/avatars...,https://files.mastodon.social/accounts/avatars...,https://files.mastodon.social/accounts/headers...,https://files.mastodon.social/accounts/headers...,261,3,2786,2021-08-07,[],[]
69,1132330,biodiversitypix,biodiversitypix@botsin.space,Biodiversity Illustrations,False,True,True,False,2020-02-29 00:00:00+00:00,<p>The Biodiversity Heritage Library has thous...,...,https://files.mastodon.social/accounts/avatars...,https://files.mastodon.social/accounts/avatars...,https://mastodon.social/headers/original/missi...,https://mastodon.social/headers/original/missi...,1235,1,11704,2021-08-07,[],"[{'name': 'Creator', 'value': 'Mark Sample', '..."
87,106487135596068032,MeeXtina,MeeXtina@writing.exchange,Truth be told,False,True,True,False,2020-09-29 00:00:00+00:00,<p>A girl who loves frogs and many other thing...,...,https://files.mastodon.social/cache/accounts/a...,https://files.mastodon.social/cache

In [64]:
pure.account(149988)

{'id': 149988,
 'username': 'dogtrax',
 'acct': 'dogtrax',
 'display_name': 'dogtrax',
 'locked': False,
 'bot': False,
 'discoverable': None,
 'group': False,
 'created_at': datetime.datetime(2017, 5, 28, 0, 0, tzinfo=tzutc()),
 'note': '<p>I teach. I write. I explore. Sometimes, I find my way back.  <a href="http://dogtrax.edublogs.org/" rel="nofollow noopener noreferrer" target="_blank"><span class="invisible">http://</span><span class="">dogtrax.edublogs.org/</span><span class="invisible"></span></a></p>',
 'url': 'https://mastodon.social/@dogtrax',
 'avatar': 'https://files.mastodon.social/accounts/avatars/000/149/988/original/9f52477716845020.png',
 'avatar_static': 'https://files.mastodon.social/accounts/avatars/000/149/988/original/9f52477716845020.png',
 'header': 'https://files.mastodon.social/accounts/headers/000/149/988/original/153e21fd36944895.jpeg',
 'header_static': 'https://files.mastodon.social/accounts/headers/000/149/988/original/153e21fd36944895.jpeg',
 'followers_